In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import numpy as np
import wandb
import torch
import torch.nn as nn

import utils
from naive_model import NaiveModel
import encoding

In [3]:
NUM_EMBEDDING = 2000

assert torch.cuda.is_available()
torch.backends.cudnn.deterministic = True
torch.set_default_tensor_type('torch.cuda.FloatTensor')

def to_device(d):
    if hasattr(d, 'cuda'):
        return d.cuda()
    return {k: v.cuda() for k, v in d.items()}

class Model(nn.Module):
    arch = 'lstm'
    def __init__(self, units, combinations=encoding.NAMES):
        super().__init__()
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        
        self.pre_lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=False, bidirectional=True)
        
        self.attention = nn.MultiheadAttention(units, num_heads=1)
        
        self.post_lstm = nn.LSTM(input_size=units, hidden_size=units, num_layers=1, batch_first=False, bidirectional=True)

        self.tasks = {}
        for combination in combinations:
            out = nn.Linear(in_features=units, out_features=encoding.class_size(combination))
            self.tasks[combination] = out
            setattr(self, encoding.class_name(combination), out)

    def isroot(self, combination):
        return any(r in combination for r in ['R1', 'R2', 'R3', 'R4'])
        
    def forward(self, x):
        embeds = self.embed(x).permute([1, 0, 2])
        
        lstm_out, (h_n0, c_n) = self.pre_lstm(embeds)
        
        left, right = torch.chunk(lstm_out, 2, dim=-1)
        lstm_out = torch.squeeze(left + right)

        attention, weights = self.attention(embeds, lstm_out, lstm_out)
        lstm_out, (h_n1, c_n) = self.post_lstm(attention + embeds)
        
        h_n = h_n0 + h_n1
#         h_n = h_n0
        return {combination: f(h_n[0] + h_n[1])
                for combination, f in self.tasks.items()}


In [8]:
def sanity():
    model = to_device(Model(100, combinations=[('B', 'T')]))
    print(model)
    with torch.no_grad():
        verbs = encoding.wordlist2numpy(["אתאקלם", "יכפיל", "בואס"])
        labels = {'B': torch.Tensor([3, 5]), 'T': torch.Tensor([2, 4])}
        verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
        tag_scores = model(verbs)
        print(f'{tag_scores[("B", "T")].shape=}')
        for combination in tag_scores:
            print(combination)
            v = tag_scores[combination]
            # print(f'{v=}')
            print(f'{labels=}')
            print()
# sanity()

In [4]:

# TEMP_PATH = 'model.pt'

#                 best_lr = 8e-4
#                 best_loss = 10
                
#                 torch.save({
#                     'state_dict': model.state_dict(),
#                     'optimizer': optimizer.state_dict(),
#                 }, TEMP_PATH)
                
#                 for i in range(1):
#                     checkpoint = torch.load(TEMP_PATH)
#                     model.load_state_dict(checkpoint['state_dict'])
#                     optimizer.load_state_dict(checkpoint['optimizer'])

def fit(model, train, test, *, epochs,  runsize, criterion, optimizer, batch_size, **_):
    train_x, train_y = train
    test_x, test_y = test
    
    assert_reasonable_initial = utils.Once(utils.assert_reasonable_initial)
    
    for epoch in range(epochs):
        train_stats = utils.Stats(model.tasks.keys())
        
        nbatches = len(train_x)
        for batch, (inputs, labels) in enumerate(zip(train_x, train_y), 1):
            model.train()

            inputs = to_device(inputs)
            labels = to_device(labels)

            outputs = model(inputs)

            losses = {combination: criterion(output.double(), labels[combination])
                      for combination, output in outputs.items()}

            loss = sum(losses.values())
            
            assert_reasonable_initial(losses, nn.CrossEntropyLoss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
#             scheduler.step()
            
            train_stats.update(loss=loss.item(),
                               batch_size=inputs.size(0),
                               d={k: (outputs[k], labels[k]) for k in outputs})

            if batch % runsize == 0 or batch == nbatches:
                model.eval()

                test_stats = utils.Stats(model.tasks.keys())
                for inputs, labels in zip(test_x, test_y):
                    inputs = to_device(inputs)
                    labels = to_device(labels)

                    with torch.no_grad():
                        outputs = model(inputs)

                    losses = {combination: criterion(output.double(), labels[combination])
                              for combination, output in outputs.items()}

                    loss = sum(losses.values())

                    test_stats.update(loss=loss.item(),
                                      batch_size=inputs.size(0),
                                      d={k: (outputs[k], labels[k]) for k in outputs})
                    
                utils.log(train_stats, test_stats, batch, nbatches, epoch)

@torch.no_grad()
def predict(model, *verbs):
    model.eval()
    verbs = encoding.wordlist2numpy(verbs)
    verbs = to_device(torch.from_numpy(verbs).to(torch.int64))
    outputs = model(verbs)
    res = {}
    # FIX: assumes no overlaps
    for combination, v in outputs.items():
        combined_index = torch.argmax(v).cpu().data.numpy()
        indices = np.unravel_index(combined_index, encoding.combined_shape(combination))
        if isinstance(combination, str):
            combination = tuple([combination])
        for k, i in zip(combination, indices):
            # assert k not in res, "Overlapping classes are not handled"
            s = k
            if k in res:
                s += "'"
            res[s] = encoding.from_category(k, i)
    if all(r in res for r in ['R1', 'R2', 'R3', 'R4']):
        res['R'] = ''.join(res[k] for k in ['R1', 'R2', 'R3', 'R4']).replace('.', '')
    return res

In [14]:
torch.manual_seed(0)
np.random.seed(0)

corpus = 'ud'
arity = 'combined'
gen = 'train'
artifact_name = f'nocontext-{gen}'
filename = f'{corpus}/{artifact_name}.tsv'  # all_verbs_shuffled
test_size = 300

artifact = wandb.Artifact(artifact_name, type='dataset')
artifact.add_file(filename)

(train_x, pre_train_y), (test_x, pre_test_y) = encoding.load_dataset_split(filename, split=test_size)

utils.shuffle_in_unison([train_x, *pre_train_y.values()])

In [ ]:
%env WANDB_SILENT true

def experiment(config, combinations=encoding.NAMES, names_str=''):
    print(config)
    
    torch.manual_seed(1)
    np.random.seed(1)
    
    train_y = utils.ravel_multi_index(pre_train_y, combinations)
    test_y = utils.ravel_multi_index(pre_test_y, combinations)
    
    train = utils.batch_xy((train_x, train_y), config['batch_size'])
    test = utils.batch_xy((test_x, test_y), config['batch_size'])
    
    model = to_device(Model(units=config['units'], combinations=combinations))  # NaiveModel.learn_from_file(filename)

    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    config.update({
        'runsize': 2 * 8192 // config['batch_size'],
        'optimizer': optimizer,
        'criterion': nn.CrossEntropyLoss(),
        'model': model,
    })
    
#     names_str = '+'.join(encoding.class_name(combination) for combination in combinations if combination not in encoding.NONROOTS)
#     if len(combinations) <= 3:
#         names_str += '_only'
    run = wandb.init(project="rootem",
                     group=f'ud',  # f'lr_units_grid_search-{arity}-{wandb.util.generate_id()}',
                     name=f"train",  # {model.arch}-{config['units']}-{config['lr']:.0e}-{config['batch_size']} f'{gen}-{arity}-{lr:.0e}',# f'{arity}-batch_{BATCH_SIZE}', # f'all-{arity}-lr_{lr:.0e}-units_{units}',
                     tags=[gen, arity, 'synthetic', 'shuffle-root', 'prefix', 'shuffle', 'batchval', 'full-root'],
                     config=config)
    with run:
        run.use_artifact(artifact)

        wandb.config.update(config, allow_val_change=True)

        if isinstance(model, nn.Module):
            wandb.watch(model)

        fit(train=train,
            test=test,
            **config
        )
        wandb.save(f"{model.arch}.h5")

    return model

%env WANDB_MODE run

config = {
    'epochs': 1,
    'test_size': test_size,
    'batch_size': 128,
    'units': 350,
    'weight_decay': 7e-4,
    'dropout': 0.2,
    'num_layers': 1,
    'lr': 1e-3,
}
model = experiment(config)


env: WANDB_SILENT=true
env: WANDB_MODE=run
{'epochs': 1, 'test_size': 300, 'batch_size': 128, 'units': 350, 'weight_decay': 0.0007, 'dropout': 0.2, 'num_layers': 1, 'lr': 0.001}


In [ ]:
def print_words(model):
    print(predict(model, 'הבריל'))
    print(predict(model, 'חגוו'))
    print(predict(model, 'עגו'))
    print(predict(model, 'צירלל'))
    print(predict(model, "השטקרפתי"))
    print(predict(model, "ישסו"))
print_words(model)